In [1]:
import numpy as np
from mpcpp import MPC_library as mpc
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import EarthLocation, solar_system_ephemeris
from astropy.coordinates import get_body_barycentric, get_body, SkyCoord, CartesianRepresentation
from jplephem.spk import SPK
from importlib import reload
reload(mpc)

<module 'mpcpp.MPC_library' from '/home/mikea/.anaconda3/envs/mpcconda37/lib/python3.7/mpcpp/MPC_library.py'>

In [2]:
def mpc_xyz(time=2458892.0, obsCode='I11'):
    ''' MPC ephemeris '''
    obs = mpc.Observatory()

    print('MPC XYZ:')
    
    ''' Heliocentric location of geocenter: '''
    helio_500_mpc = obs.getObservatoryPosition(obsCode='500', jd_utc=time, old=False) * u.au
    print('Heliocentric location of  geocenter:  {}'.format(helio_500_mpc.to(u.km)))

    ''' Heliocentric location of observatory: '''
    helio_OBS_mpc = obs.getObservatoryPosition(obsCode=obsCode, jd_utc=time, old=False) * u.au
        
    ''' Geocentric location of observatory '''
    geo_OBS_mpc = helio_OBS_mpc - helio_500_mpc
    print(' Geocentric  location of observatory: {}'.format(geo_OBS_mpc.to(u.km)))

    print('Heliocentric location of observatory: {}\n'.format(helio_OBS_mpc.to(u.km)))
        
    return helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc

In [3]:
def apy_xyz(time=2458892.0, site_name='gemini_south'):
    ''' Astropy ephemeris '''
    print('Astropy XYZ:')
    t_jd = Time(time, format='jd')
    
    ''' Barycentric location of geocenter: '''
    bary_500_apy = get_body_barycentric('earth', t_jd, ephemeris='de430').xyz

    ''' Barycentric location of the Sun: '''
    bary_Sun_apy = get_body_barycentric('sun', t_jd, ephemeris='de430').xyz

    ''' Heliocentric location of geocenter: '''
    helio_500_apy = bary_500_apy - bary_Sun_apy
    print('Heliocentric location of  geocenter:  {}'.format(helio_500_apy))

    ''' Geocentric location of observatory: '''
    loc_0 = EarthLocation.of_site(site_name)        # at reference time
    loc_t, _ = loc_0.get_gcrs_posvel(obstime=t_jd)  # at correct time
    geo_OBS_apy = loc_t.xyz                         # in km, not AU
    print(' Geocentric  location of observatory: {}'.format(geo_OBS_apy))

    ''' Heliocentric location of observatory: '''
    helio_OBS_apy = helio_500_apy + geo_OBS_apy
    print('Heliocentric location of observatory: {}\n'.format(helio_OBS_apy))
    
    return helio_500_apy, geo_OBS_apy, helio_OBS_apy

In [4]:
def diff_apy_mpc(time=2458892.0, obsCode='I11', site_name='gemini_south'):
    ''' Comparing Astropy and MPC '''
    print('v'*30, site_name, obsCode, 'v'*30)

    helio_500_apy, geo_OBS_apy, helio_OBS_apy = apy_xyz(time, site_name)
    helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc = mpc_xyz(time, obsCode)

    print('Discrepancies (Astropy - MPC):')
    helio_500_diff = helio_500_apy - helio_500_mpc
    helio_500_diff_abs = (np.sum(helio_500_diff ** 2) ** 0.5).to(u.m)
    print('Heliocentric location of  geocenter:  {0:}'.format(helio_500_diff_abs))

    geo_OBS_diff = geo_OBS_apy - geo_OBS_mpc
    geo_OBS_diff_abs = (np.sum(geo_OBS_diff ** 2) ** 0.5).to(u.m)
    print(' Geocentric  location of observatory: {0:}'.format(geo_OBS_diff_abs))

    helio_OBS_diff = helio_OBS_apy - helio_OBS_mpc
    helio_OBS_diff_abs = (np.sum(helio_OBS_diff ** 2) ** 0.5).to(u.m)
    print('Heliocentric location of observatory: {0:}'.format(helio_OBS_diff_abs))
    
    print('^'*30, site_name, obsCode, '^'*30, '\n')
    
    return helio_500_apy, geo_OBS_apy, helio_OBS_apy, helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc

In [5]:
time_jd = 2458892.0
helio_500_apy, geo_OBS_apy, helio_OBS_apy, helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc = diff_apy_mpc(time_jd, 'I11', 'gemini_south')
#helio_500_apy, geo_OBS_apy, helio_OBS_apy, helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc = diff_apy_mpc(time_jd, 'T14', 'CFHT')  # Mauna Kea-UH/Tholen NEO Follow-Up (CFHT)
#helio_500_apy, geo_OBS_apy, helio_OBS_apy, helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc = diff_apy_mpc(time_jd, 'T09', 'Subaru')  # Mauna Kea-UH/Tholen NEO Follow-Up (Subaru)
#helio_500_apy, geo_OBS_apy, helio_OBS_apy, helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc = diff_apy_mpc(time_jd, 'G83', 'Large Binocular Telescope')  # Mt. Graham-LBT
#helio_500_apy, geo_OBS_apy, helio_OBS_apy, helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc = diff_apy_mpc(time_jd-182, 'G83', 'Large Binocular Telescope')  # Mt. Graham-LBT
#helio_500_apy, geo_OBS_apy, helio_OBS_apy, helio_500_mpc, geo_OBS_mpc, helio_OBS_mpc = diff_apy_mpc(time_jd-91, 'G83', 'Large Binocular Telescope')  # Mt. Graham-LBT

vvvvvvvvvvvvvvvvvvvvvvvvvvvvvv gemini_south I11 vvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
Astropy XYZ:
Heliocentric location of  geocenter:  [-1.17854006e+08  8.16169879e+07  3.53807465e+07] km
 Geocentric  location of observatory: [-1800872.4070786  -5217154.70911297 -3191412.52214763] m
Heliocentric location of observatory: [-1.17855807e+08  8.16117707e+07  3.53775551e+07] km

MPC XYZ:
Heliocentric location of  geocenter:  [-1.17854006e+08  8.16169879e+07  3.53807465e+07] km
 Geocentric  location of observatory: [-1800.85214402 -5217.13544829 -3191.38491209] km
Heliocentric location of observatory: [-1.17855807e+08  8.16117708e+07  3.53775551e+07] km

Discrepancies (Astropy - MPC):
Heliocentric location of  geocenter:  0.057528397285059554 m
 Geocentric  location of observatory: 39.29231116361542 m
Heliocentric location of observatory: 39.24096852400452 m
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ gemini_south I11 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^ 



In [6]:
print(helio_500_mpc)

[-0.78780537  0.54557587  0.23650568] AU


In [7]:
#EarthLocation.get_site_names()

In [8]:
'''geocentric location of observatory'''
geo_OBS_horizons = [1.800856351642355E+03, 5.217132743849016E+03, 3.191382310369976E+03] * u.km
#geo_OBS_horizons = [3.897755155906439E-05, -9.519648682208122E-06, -1.445320321166943E-05] * u.au
#geo_OBS_horizons = [3.897607996384405E-05, -9.524410913145084E-06, -1.445330040715949E-05] * u.au
#geo_OBS_horizons = [3.047176273153229E-05, 1.896779432438119E-05, -2.297248162195744E-05] * u.au
#geo_OBS_horizons = [-3.035225384512615E-05, -1.929568746151384E-05,-2.285733119326494E-05] * u.au
#geo_OBS_horizons = [1.908870978408335E-05, -3.041274366209686E-05, -2.295052590013195E-05] * u.au
'''Astropy-Horizons'''
print((geo_OBS_apy + geo_OBS_horizons).to(u.m))
#print((np.sum((geo_OBS_apy + geo_OBS_horizons) ** 2) ** 0.5).to(u.m))
'''MPC-Horizons'''
print((geo_OBS_mpc + geo_OBS_horizons).to(u.m))
#print((np.sum((geo_OBS_mpc + geo_OBS_horizons) ** 2) ** 0.5).to(u.m))

[-16.05543625 -21.96526395 -30.21177765] m
[ 4.20761912 -2.70444457 -2.60172258] m


In [9]:
'''heliocentric location of geocentre'''
helio_500_horizons = [7.878053705097503E-01, -5.455758662127957E-01, -2.365056822060531E-01] * u.au
'''Astropy-Horizons:'''
print((helio_500_apy + helio_500_horizons).to(u.m))
'''MPC-Horizons:'''
print((helio_500_mpc + helio_500_horizons).to(u.m))

[0.03626943 0.02627075 0.01086295] m
[ 0.00111278 -0.01551253 -0.00725385] m


In [10]:
'''heliocentric location of observer'''
helio_OBS_horizons = [7.878174084909276E-01, -5.455409918344410E-01, -2.364843491329590E-01] * u.au
#helio_OBS_horizons = [7.878443480613093E-01, -5.455853858614780E-01, -2.365201354092648E-01] * u.au
#helio_OBS_horizons = [7.878443465897141E-01, -5.455853906237088E-01, -2.365201355064603E-01] * u.au
#helio_OBS_horizons = [7.878358422724819E-01, -5.455568984184713E-01, -2.365286546876751E-01] * u.au
#helio_OBS_horizons = [-7.890835808625358E-01, 5.828815097032926E-01, 2.526647001858481E-01] * u.au
#helio_OBS_horizons = [-6.289870474582483E-01, -7.009816282331524E-01, -3.038845132122471E-01] * u.au
'''Astropy-Horizons:'''
print((helio_OBS_apy + helio_OBS_horizons).to(u.m))
'''MPC-Horizons:'''
print((helio_OBS_mpc + helio_OBS_horizons).to(u.m))

[-16.01915061 -21.93899453 -30.20091355] m
[ 4.20874424 -2.71995698 -2.60897765] m


In [11]:
print((helio_500_apy + geo_OBS_mpc + helio_OBS_horizons).to(u.m))

[ 4.24391031 -2.6781708  -2.5908649 ] m
